# Training Trajectory Corrector

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from glob import glob
# from flow_predictor import flow_predictor
from tqdm import tqdm
# from pathgen import training_twosteps

mps_device = torch.device("cuda:1")

plt.close('all')
from constants.filepath import PROJECT_PATH
import os

In [2]:
residuals = np.load(os.path.join(PROJECT_PATH, 'model_data', 'residuals.npz'))['residuals']
command_data = np.load(os.path.join(PROJECT_PATH, 'model_data', 'command_data.npz'))['command_data']
analytical_data = np.load(os.path.join(PROJECT_PATH, 'model_data', 'analytical_data.npz'))['analytical_data']


input_data = np.stack((command_data, analytical_data), axis=1)
# input_features = torch.tensor(analytical_data, dtype=torch.float32, device=mps_device).view(-1, 1)
input_features = torch.tensor(input_data, dtype=torch.float32, device='cpu')
target_residuals = torch.tensor(residuals, dtype=torch.float32, device='cpu').view(-1, 1)

print("input_features shape:", input_features.shape)
print("target_residuals shape:", target_residuals.shape)

input_features shape: torch.Size([3162222, 2])
target_residuals shape: torch.Size([3162222, 1])


In [3]:
class BigManBlastoiseDataset(torch.utils.data.Dataset):
    def __init__(self, input_features, target_residuals):
        self.data_length = input_features.shape[0]
        self.input_features = input_features
        self.target_residuals = target_residuals
    def __getitem__(self, idx):
        return self.input_features[idx], self.target_residuals[idx]
    def __len__(self):
        return self.data_length

In [ ]:
dataset = BigManBlastoiseDataset(input_features, target_residuals)
# 524288
# 1048576
data_loader = torch.utils.data.DataLoader(dataset, batch_size=524288, shuffle=True, num_workers=16)

data_batch, data_label = next(iter(data_loader))
print(data_batch.shape)
print(data_label.shape)
print(data_batch.device)

torch.Size([524288, 2])
torch.Size([524288, 1])
cpu


In [5]:
class ResidualModel(torch.nn.Module):
    def __init__(self, fc_dim_list = [2, 64, 512, 1024, 512, 64, 1]):
        super(ResidualModel, self).__init__()

        self.sequence = []
        for i in range(1,len(fc_dim_list)):
            self.sequence.append(torch.nn.Linear(fc_dim_list[i-1], fc_dim_list[i]))
            if i < len(fc_dim_list)-1:
                self.sequence.append(torch.nn.ReLU())
        self.sequence = torch.nn.Sequential(*self.sequence)

    def forward(self, x):
        return self.sequence(x)

In [6]:
model = ResidualModel()
model.to('cuda:1')
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Training loop
epochs = 100
model.train()

# input_features = torch.tensor(input_features).to('cuda:1')
# residuals = torch.tensor(residuals).to('cuda:1')

print("Training model...")
for epoch in tqdm(range(epochs)):
    for batched_input_features, batched_residuals in data_loader:
        batched_input_features = batched_input_features.to('cuda:1')
        batched_residuals = batched_residuals.to('cuda:1')
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(batched_input_features)
        loss = criterion(predictions, batched_residuals)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    # print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Training model...


 10%|█         | 10/100 [00:59<08:59,  5.99s/it]

Epoch [10/100], Loss: 0.5828


 20%|██        | 20/100 [02:00<08:01,  6.01s/it]

Epoch [20/100], Loss: 0.5086


...

Epoch [80/100], Loss: 0.3633


 90%|█████████ | 90/100 [09:02<01:00,  6.06s/it]

Epoch [90/100], Loss: 0.3403


100%|██████████| 100/100 [10:02<00:00,  6.03s/it]

Epoch [100/100], Loss: 0.3511


In [7]:
# Example: Run model inference (optional, based on your needs)
model.eval()

#%%

test = training_twosteps(flowrate_magnitudes = [[0.75, 0.001], [0.2, 0.001], [0.48, 0.001]],
                         flowrate_times = [[0.0001, 8], [15, 20], [22, 33]],
                         beadwidth_magnitudes = [[]],
                         beadwidth_times = [[]],
                         dt = 0.001,
                         tmax=40)
test_t_input, test_Q_input, test_W_input = test.pathgen()
test_t, test_W, test_Q_com, test_Q_out = flow_predictor(test_t_input, test_Q_input, test_W_input)

test_analytical = test_Q_out * 1e9
test_command = test_Q_com * 1e9
test_combined = np.stack((test_command, test_analytical), axis=1)
test_input = torch.tensor(test_combined, dtype=torch.float32, device=mps_device)

with torch.no_grad():
    output = model(test_input) * 1e-9
    test_result = (output).cpu().numpy().reshape(-1)
    
plt.figure()
plt.plot(test_t, test_Q_com, label='Input', color='black', linestyle='--')
# plt.plot(test_accel.ts, test_accel.sim_Q_out, label = 'Simulated Data', color = 'red')
plt.plot(test_t, test_Q_out, label = 'Analytical Data', color = 'blue')
plt.plot(test_t, test_result, label='Residual', color='magenta')
plt.plot(test_t, test_Q_out + test_result, label='Total', color='green')

plt.legend()


NameError: name 'training_twosteps' is not defined

In [8]:
torch.save(model.state_dict(), 'trajectory_correction_DNN_v0.pth')